# Stage 2

- has **`location_raw`**: True
- vars one-hot encoded: True
- var label-encoded: False
- oversampled: False

In [7]:
import pickle

import humanize
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.externals.joblib import parallel_backend
from sklearn.model_selection import train_test_split

%matplotlib inline
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
X_train = pickle.load(open('./data/stage2-train.pkl', 'rb'))
y_train = X_train.pop('stop_outcome')
X_test = pickle.load(open('./data/stage2-test.pkl', 'rb'))
y_test = X_test.pop('stop_outcome')

print('Row counts:\n\ttrain: {}\n\ttest: {}'.format(X_train.shape[0], X_test.shape[0]))

Row counts:
	train: 222186
	test: 55547


In [3]:
# features = pickle.load(open('./data/stage2-oh_encoded.pkl', 'rb'))
# labels = features.pop('stop_outcome')

In [4]:
# Split the 'features' and 'labels' data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(features,
#                                                     labels, 
#                                                     test_size=0.2, 
#                                                     random_state=0)
# final_test_features = pickle.load(open('./data/final_test_set.pkl', 'rb'))
# final_test_outcomes = final_test_features.pop('stop_outcome')
# print('Final test set row count: {}'.format(final_test_features.shape[0]))

# X_train = features
# y_train = labels
# X_test = final_test_features
# y_test = final_test_outcomes

# Simple Linear Classifier

In [5]:
from sklearn import linear_model

clf_sgd = linear_model.SGDClassifier()
clf_sgd.fit(X_train, y_train)
clf_sgd.score(X_test, y_test)

/home/pato/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.68210704448485071

# VotingClassifier

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier

clf1 = RandomForestClassifier(n_jobs=8, verbose=3, random_state=0)
clf2 = GaussianNB()
clf3 = DecisionTreeClassifier(random_state=0)
clf4 = GradientBoostingClassifier(verbose=3, random_state=0)

eclf = VotingClassifier(estimators=[('rf', clf1), ('gnb', clf2), ('dt', clf3), ('gb', clf4)],
                        voting='soft')
eclf = eclf.fit(X_train, y_train)
print('eclf score: {}'.format(eclf.score(X_test, y_test)))

building tree 3 of 10
building tree 4 of 10building tree 1 of 10building tree 7 of 10building tree 2 of 10building tree 6 of 10building tree 8 of 10building tree 5 of 10






building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=8)]: Done   3 out of  10 | elapsed:    0.6s remaining:    1.3s
[Parallel(n_jobs=8)]: Done   7 out of  10 | elapsed:    0.7s remaining:    0.3s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.8s finished


      Iter       Train Loss   Remaining Time 
         1      281155.7293            1.56m
         2      266048.4388            1.54m
         3      254042.7355            1.50m
         4      244288.8940            1.48m
         5      236316.8425            1.46m
         6      229734.5819            1.45m
         7      224223.7344            1.45m
         8      219641.5698            1.44m
         9      215744.3363            1.44m
        10      212425.1033            1.43m
        11      209634.3275            1.42m
        12      207179.6033            1.41m
        13      205104.2469            1.39m
        14      203253.4101            1.38m
        15      201609.5064            1.37m
        16      200185.8394            1.36m
        17      198971.0307            1.35m
        18      197894.2861            1.34m
        19      196914.3399            1.32m
        20      196078.0901            1.31m
        21      195294.3866            1.30m
        2

[Parallel(n_jobs=8)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   7 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished


eclf score: 0.65782130448089


In [7]:
eclf.estimators_

[RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=8,
             oob_score=False, random_state=0, verbose=3, warm_start=False),
 GaussianNB(priors=None),
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=0,
             splitter='best'),
 GradientBoostingClassifier(criterion='friedman_mse', init=None,
               learning_rate=0.1, loss='deviance', max_depth=3,
               max_features=None, max_leaf_nodes=None,
               min_impu

# GradientBoostingClassifier

In [8]:
from sklearn.ensemble import GradientBoostingClassifier


gbc = GradientBoostingClassifier(random_state=0)
with parallel_backend('threading'):
    gbc.fit(X_train, y_train)
print('{}'.format(gbc.score(X_test, y_test)))

0.7095432696635282


## GradientBoostingClassifier (Tuned)


In [9]:
gbc_tuned = GradientBoostingClassifier(
    learning_rate=0.0983,
    max_depth=6,
    max_features=len(list(X_train.columns.values)),
    subsample=0.9,
    verbose=3,
    random_state=0,
)
with parallel_backend('threading'):
    gbc_tuned.fit(X_train, y_train)
print('{}'.format(gbc_tuned.score(X_test, y_test)))

      Iter       Train Loss      OOB Improve   Remaining Time 
         1      251946.7001        2052.8657            5.50m
         2      237414.4177        1575.6063            5.49m
         3      226146.2405        1280.7294            5.44m
         4      216676.8115        1016.1126            5.38m
         5      209071.7951         835.2777            5.31m
         6      202845.3294         693.5397            5.27m
         7      197641.6092         569.6656            5.20m
         8      193158.3705         474.1783            5.19m
         9      189489.3661         400.0201            5.14m
        10      186065.2030         343.1777            5.10m
        11      183306.9784         299.3411            5.07m
        12      180943.3302         233.9014            5.02m
        13      179019.0402         232.2621            5.00m
        14      177193.4287         193.0601            4.97m
        15      175452.1221         159.1138            4.92m
       